In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets

In [2]:
def splitTrainTest(dataset, y, split=0.75):
    """ 
    Separating the dataset into 2 parts: Training Dataset (to train the model) & Test Dataset (to evaluate the performance of the model)
    The rows assigned to each dataset are randomly selected (to ensure that the model is objective).
    randrange() generate a random integer in the range between 0 and the size of the list.

    Parameters:
        dataset: The dataset to split as a list of lists
        split: Split percentage. (default split = 60%) --> A 60/40 for train/test
        
    Returns:
        train: 60% of the dataset
        test: The rows that remain in the copy of the dataset are then returned as the test dataset. (40%)
    """

    #calculate how many rows the training set requires
    xTrain = pd.DataFrame()
    yTrain = pd.DataFrame()
    trainSize = split * len(dataset)
    datasetCopy = dataset.copy()
    yCopy = y.copy()

    #add index column
    datasetCopy.reset_index(inplace=True)
    datasetCopy = datasetCopy.rename(columns={"index": "index"})
    yCopy.reset_index(inplace=True)
    yCopy = yCopy.rename(columns={"index": "index"})

    idxRan = len(datasetCopy)
    while len(xTrain) < trainSize: # while until the train dataset contains the target number of rows.
        randomIndex = np.random.choice(datasetCopy.index, 1, replace=False) #select random rows

        datasetCopy = datasetCopy.drop(datasetCopy["index"][randomIndex]) #remove random rows from the datasetCopy
        yCopy = yCopy.drop(yCopy["index"][randomIndex]) #remove random rows from the datasetCopy

        xTrain = pd.concat([xTrain, dataset.loc[randomIndex]]) #add rows to train dataset
        yTrain = pd.concat([yTrain, y.loc[randomIndex]]) #add rows to train dataset
        idxRan = idxRan - 1
    
    datasetCopy = datasetCopy.drop(labels=["index"], axis=1)
    yCopy = yCopy.drop(labels=["index"], axis=1)
    return xTrain, datasetCopy, yTrain, yCopy

In [3]:
def getData():
    """
    Args:
        file (str, optional): Location of Iris.csv file. Defaults to 'Iris.csv'.
        binary_version (bool, optional): Select if binary labels are used. Defaults to True.
            target variable will select the positive label.

    Returns:
        array: Features and labels in the las column.
        dict: Encodig of labels to string.
    """
    iris = datasets.load_iris()
    df = pd.DataFrame(iris.data, columns = iris.feature_names)
    y = pd.DataFrame(iris.target, columns = ["Target"])

    xTrain, xTest, yTrain, yTest = splitTrainTest(df,y)

    return xTrain, xTest, yTrain, yTest

In [4]:
class StandarScaler:
    """ 
    Standardize features by removing the mean and scaling to unit variance. 
    z = (x - MEAN) / DESV EST 
    """
    def __init__(self):
        pass

    def fit(self, X):
        X = pd.DataFrame(X)
        self.mean = X.mean(axis = 0).to_numpy()
        self.std = X.std(axis = 0).to_numpy()
        
    def transform(self, X):
        X -= self.mean
        X /= self.std
        return X 
    
    def fitTransform(self, X):
        self.fit(X)
        df = self.transform(X)
        return df 

In [5]:
def oneHot(y):
    """
    Converts the training data into a series of ones and zeros for the classes given.
    """
    yEncoded = np.zeros(shape=(y.size, int(y.max()[0])+1))
    y = y.Target.tolist()
    for i in range(len(y)): # rows
        yEncoded[i,y[i]] = 1 # sub fila, sub columna que diga y
    return yEncoded

In [36]:
class MultiLogisticReg():
    def __init__(self):
        pass

    def softMaxFunc(self, z):
        # sigm = []
        # for i in range(len(z)):
        #     lista = np.exp(z[i])/sum(np.exp(z[i]))
        #     sigm.append(lista.tolist())
        sigm = np.exp(z) / np.sum(np.exp(z), axis = 1, keepdims = True)
        return sigm
    
    def getWeights(self,X,y):
        cantTargs = np.shape(y)[1]
        feats = np.shape(X)[1] # columnas 
        weight = np.random.rand(feats, cantTargs) # array FEATURE * CLASES
        b = np.ones(cantTargs)
        return weight, b

    def fit(self,X,y, rounds = 1000, lRate=0.1):
        """
        weighted sum of the inputs plus a bias term 
        For multiclass problems, only ‘newton-cg’, ‘sag’, ‘saga’ and ‘lbfgs’ 
        handle multinomial loss; SOFTMAX
        """

        self.weight, self.bias = self.gradientDescent(rounds, lRate, X, y)
            
    def gradientDescent(self, rounds, lRate, X, y):
        weight, bias = self.getWeights(X, y)
        losses = []
        rows = X.shape[0]
        count = 0

        while count < rounds:
            z = np.dot(X, weight) + bias # (w·x)
            yG = self.softMaxFunc(z) # yˆ = σ(w·x) Ya son probs
            gradient = 1/rows * np.dot(X.T, (yG - y))
            #updtate a lo weights
            weight -= lRate * gradient
            bias -= lRate * np.sum((yG - y))
            count += 1 

            loss = self.lossFunction(z, y) # loss entre ygorrito y y train
            losses.append(loss)

        self.lossSteps = losses
        return weight, bias

    def lossFunction(self, z, y):
        """
        Calculate cross-entropy loss
        The loss increases as the predicted probability diverge from the actual label.

        Parameters:
        yG:
        y:

        Returns:
        loss: Average cross entropy loss
        """

        # Y must be one-hot encoded
        rows = y.shape[0]
        loglLoss = 1/rows * (np.trace(np.dot(z, y.T)) + np.sum(np.log(np.sum(np.exp(z), axis=1))))
        return loglLoss

    def predict(self, X):
        z = np.dot(X, self.weight) + self.bias
        probs = self.softMaxFunc(z)
        predictions = np.argmax(probs, axis=1)
        return predictions

    def predictProba(self, X):
        z = np.dot(X, self.weight) + self.bias
        probs = self.softMaxFunc(z)
        return probs

In [64]:
xTrain, xTest, yTrain, yTest = getData()

In [65]:
ss = StandarScaler()

xTrain = ss.fitTransform(xTrain)
xTest = ss.fitTransform(xTest)

In [66]:
yTrain = oneHot(yTrain)

In [67]:
mlr = MultiLogisticReg()

In [68]:
mlr.fit(xTrain, yTrain)

In [69]:
yPred = mlr.predict(xTest)
yProbs = mlr.predictProba(xTest)

In [70]:
yPred = yPred.tolist()
yTest = yTest.Target.tolist()

In [51]:
def accuracy(yTest, yPred):
    """
    Classification accuracy is a ratio of the number of correct predictions out of all predictions that were made.
    accuracy = TP+TN / FP+FN+TP+TN
    """
    # # -- OPCION 1 --
    # #TRUE positive
    # TP = sum((yTest == 1) & (yPred == 1))

    # #FALSE positive
    # FP = sum((yTest == 0) & (yPred == 1))

    # #FALSE positive
    # FN = sum((yTest == 1) & (yPred == 0))

    # #TRUE negative
    # TN = sum((yTest == 0) & (yPred == 0))
    
    # return (TP + TN)/(FP + FN + TP + TN)

    # -- OPCION 2 --
    correct = 0
    for i in range(len(yTest)):
        if yTest[i] == yPred[i]:
            correct += 1
    return correct / float(len(yTest)) 


In [53]:
accuracy(yTest, yPred)

0.8108108108108109

In [55]:
def precision(yTest, yPred):
    """
    Precision is the ratio between the true positives and all the points that are classified as positives.
    precision = TP/(TP + FP)
    """

    #TRUE positive
    TP = sum((yTest == 1) & (yPred == 1))

    #FALSE positive
    FP = sum((yTest == 0) & (yPred == 1))

    return TP / (TP + FP)

    # -- OPCION 2 --
    tp2 = 0
    fp2 = 0
    for i in range(len(yTest)):
        if yTest[i] == yPred[i]:
            tp2 += 1
            
        if yTest[i] != yPred[i]:
            fp2 += 1 
    # return float(tp2 / (tp2 + fp2))

In [56]:
def recall(yTest, yPred):
    """
    Recall is the measure of the model correctly identifying true positives. 
    recall = TP/(TP + FN)
    """

    #TRUE positive
    TP = sum((yTest == 1) & (yPred == 1))

    #FALSE negative
    FN = sum((yTest == 1) & (yPred == 0))

    return TP / float(TP + FN)

    # -- OPCION 2 --
    tp2 = 0
    fn2 = 0
    for i in range(len(yTest)):
        if yTest[i] == yPred[i]:
            tp2 += 1
            
        if yTest[i] != yPred[i]:
            fn2 += 1
    # return float(tp2 / (tp2 + fn2))

In [57]:
def F1score(yTest, yPred):
    """
    F1 score is the combination of precision and recall. 
    F1 score = (2 * precision * recall) / (precision + recall)
    """

    Precision = precision(yTest, yPred)
    Recall = recall(yTest, yPred)

    return (2 * Precision * Recall) / (Precision + Recall)

In [71]:
from sklearn.metrics import confusion_matrix
cnf_matrix = confusion_matrix(yTest, yPred)

FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
TP = np.diag(cnf_matrix)
TN = cnf_matrix.sum() - (FP + FN + TP)
FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)
# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)
# Overall accuracy for each class
ACC = (TP+TN)/(TP+FP+FN+TN)

#accuracy
accuracy = (TP + TN)/(FP + FN + TP + TN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1score = (2 * precision * recall) / (precision + recall)

print("accuracy", accuracy)
print("precision", precision)
print("recall", recall)
print("f1score", f1score)

accuracy [0.97297297 0.86486486 0.89189189]
precision [0.90909091 1.         0.76470588]
recall [1.         0.64285714 1.        ]
f1score [0.95238095 0.7826087  0.86666667]
